In [1]:
# Install GDAL and Geopandas
# !pip install geopandas -q
# !pip install descartes -q

# for rasters
# !pip install georasters -q

In [2]:
# for download and decompress the dataset
import os, zipfile, glob, io, requests

# control
import numpy as np

import pandas as pd
import geopandas as gpd
from osgeo import gdal
import georasters as gr

#plotting
import matplotlib.pyplot as plt


# **Arangements**

In [3]:
# creating folder to store information
%mkdir clip_shapefiles
%mkdir outcomes

mkdir: cannot create directory ‘clip_shapefiles’: File exists
mkdir: cannot create directory ‘outcomes’: File exists


### Function: to fetch and create a list of raster

In [4]:
# Function to clip and save rasters
def clip_raster(list_raster, inshp, wordclim_dataset):
  ''' Get a clipped raster from the Worldclim data for specified countries:
  - list_raster : geotiff fetched from Worldclim
  - inshp : path clipped shapefile with the name of the country
  - wordclim_dataset: name of the dataset for outraster file name '''
  for i in list_raster:
    name, raster = i
    outraster = './outcomes/' + str(wordclim_dataset) + '_' + str.upper(name) + '.tif'
    dsClip = gdal.Warp(outraster, raster, cutlineDSName=inshp, cropToCutline=True, dstNodata=np.nan)
  
  # print the results
  print('%2d Rasters were clipped and stored as %s.' %(len(list_raster), wordclim_dataset))

### Function: to clip a list rasters

In [5]:
# Function to clip rasters
def list_rasters(url):
    ''' Fetcht the raster datasets by url, download it and 
    subset 2 rasters:
    - url : raster dataset url from Worldclim'''

    # Fetching geotif form worldclim
    fetching_url = f'! wget {url} -q'
    os.system(fetching_url)

    # Extracting the data
    zip_name = url.rsplit('/')[-1]
    with zipfile.ZipFile(zip_name, 'r') as temp:
        temp.extractall('temp_folder')

    # removing zip folder
    folder = f'rm {zip_name}'
    os.system(folder)

    # path to the folder
    files = glob.glob(os.path.join('./temp_folder', '*tif'))
    files.sort(key=lambda x:int(x[x.rfind('_')+1:-4]))

    ###### list of tiff files ######
    # Chosing BIO1 = Annual Mean Temperature (files[0])
    # Chosing BIO12 = Annual Precipitation (files[11])
    subset = files[0], files[11]

    # Rename and arrange the information
    names = ['BIO'+ i[i.rfind('_')+1:-4] for i in subset]

    # merging all the metadata
    list_raster = list(zip(names, [gdal.Open(f) for f in subset]))
    list_raster.sort(key=lambda x:int(x[0][3:]))

    # removing folder
    os.system(f'rm -r temp_folder')

    # print the results
    print('The list contains %2d raster files.' %(len(list_raster)))

    return list_raster

# **Belize and Colombi Shapefiles boundaries**

In [6]:
# importing the data
world_df = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# subseting columns
world_df = world_df[['continent','name', 'iso_a3','geometry']]

# croppping boundaries and save shapefiles
for i in ['Colombia', 'Belize']:
    temp = world_df.loc[world_df.name == i, ].reset_index(drop=True)
    temp.to_file("./clip_shapefiles/" + i.lower() + ".shp")

# load the shapefiles from cropping
colombia = './clip_shapefiles/colombia.shp'
belize = './clip_shapefiles/belize.shp'

# **Getting clipped tiffs**

In [7]:
data_url = {'worldclim10m' : 'https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_bio.zip',
            'worldclim5m' : 'https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_bio.zip',
            'worldclim2_5m' : 'https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_bio.zip',
            'worldclim30s' : 'https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_bio.zip'}

## **1. Wordclim 10m**

In [8]:
# fetching and subsetting rasters
tiff_bioclim10 = list_rasters(data_url['worldclim10m'])

# Colombia
clip_raster(tiff_bioclim10, colombia, 'Colombia-BioClim10')

# Belize
clip_raster(tiff_bioclim10, belize, 'Belize-BioClim10')

The list contains  2 raster files.
 2 Rasters were clipped and stored as Colombia-BioClim10.
 2 Rasters were clipped and stored as Belize-BioClim10.


## **2. Wordclim 5m**

In [9]:
# fetching and subsetting rasters
tiff_bioclim5 = list_rasters(data_url['worldclim5m'])

# Colombia
clip_raster(tiff_bioclim5, colombia, 'Colombia-BioClim5')

# Belize
clip_raster(tiff_bioclim5, belize, 'Belize-BioClim5')

The list contains  2 raster files.
 2 Rasters were clipped and stored as Colombia-BioClim5.
 2 Rasters were clipped and stored as Belize-BioClim5.


## **3. Wordclim 2.5m**

In [10]:
# fetching and subsetting rasters
tiff_bioclim2_5 = list_rasters(data_url['worldclim2_5m'])

# Colombia
clip_raster(tiff_bioclim2_5, colombia, 'Colombia-BioClim2_5')

# Belize
clip_raster(tiff_bioclim2_5, belize, 'Belize-BioClim2_5')

The list contains  2 raster files.
 2 Rasters were clipped and stored as Colombia-BioClim2_5.
 2 Rasters were clipped and stored as Belize-BioClim2_5.




---



## **4. Wordclim 30s** (9 gb, could take a bunch of time)

In [11]:
# fetching and subsetting rasters
tiff_bioclim30s = list_rasters(data_url['worldclim30s'])

# Colombia
clip_raster(tiff_bioclim30s, colombia, 'Colombia-BioClim30s')

# Belize
clip_raster(tiff_bioclim30s, belize, 'Belize-BioClim30s')

The list contains  2 raster files.
 2 Rasters were clipped and stored as Colombia-BioClim30s.
 2 Rasters were clipped and stored as Belize-BioClim30s.


## **5. Exporting clips for GDrive**

In [14]:
# compressing the data in zip files
!zip -r clip_shapefilesc.zip clip_shapefiles/
!zip -r clip_tiffWorldclim.zip outcomes/

  adding: clip_shapefiles/ (stored 0%)
  adding: clip_shapefiles/belize.prj (deflated 17%)
  adding: clip_shapefiles/colombia.shp (deflated 8%)
  adding: clip_shapefiles/colombia.prj (deflated 17%)
  adding: clip_shapefiles/colombia.dbf (deflated 77%)
  adding: clip_shapefiles/belize.shx (deflated 46%)
  adding: clip_shapefiles/belize.dbf (deflated 77%)
  adding: clip_shapefiles/belize.shp (deflated 27%)
  adding: clip_shapefiles/belize.cpg (stored 0%)
  adding: clip_shapefiles/colombia.cpg (stored 0%)
  adding: clip_shapefiles/colombia.shx (deflated 45%)
  adding: outcomes/ (stored 0%)
  adding: outcomes/Belize-BioClim30s_BIO12.tif (deflated 78%)
  adding: outcomes/Colombia-BioClim2_5_BIO1.tif (deflated 65%)
  adding: outcomes/Colombia-BioClim5_BIO12.tif (deflated 75%)
  adding: outcomes/Colombia-BioClim10_BIO1.tif (deflated 58%)
  adding: outcomes/Belize-BioClim10_BIO12.tif (deflated 43%)
  adding: outcomes/Colombia-BioClim30s_BIO1.tif (deflated 79%)
  adding: outcomes/Belize-BioClim